In [3]:
import sys
sys.path.append('/Users/artemveshkin/dev/lct_2024_task_10/address')

In [4]:
from address_parser import AddressParser
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import os
import json
from collections import defaultdict

In [5]:
DATA_DIR = Path('../data/')

In [6]:
parser = AddressParser()

In [7]:
with open(DATA_DIR / 'mingkh_addresses.json', 'r') as fh:
    mingkh_addresses = json.load(fh)

In [8]:
mingkh_addresses[0]

{'mingkh_id': 1336199, 'mingkh_address': 'ул. лобачевского, 120 корпус 1'}

In [9]:
def try_fix_mingkh_address(address):
    address = address.replace('-й', '')
    for street_type in ('пер.', 'проезд', 'б-р', 'просек'):
        if street_type in address:
            address = street_type + ' ' + address.replace(street_type, '')
    return address

In [15]:
for mingkh_item in tqdm(mingkh_addresses):
    address = mingkh_item['mingkh_address']
    parsed_address = parser(address)[0]
    unparsed = parsed_address['unparsed_parts']
    if len(unparsed) > 0:
        fixed_address = try_fix_mingkh_address(address)
        parsed_address = parser(fixed_address)[0]
        unparsed = parsed_address['unparsed_parts']
        if len(unparsed) > 0:
            continue
    mingkh_item['parsed_mingkh_address'] = parsed_address

  0%|          | 0/32716 [00:00<?, ?it/s]

In [44]:
list(filter(lambda x: x['mingkh_id'] == 203928, mingkh_addresses))[0]

{'mingkh_id': 203928,
 'mingkh_address': 'нагатинская наб., 48/2',
 'parsed_mingkh_address': {'settlement': 'нагатинская',
  'street_type': 'наб',
  'house': '48/2',
  'unparsed_parts': []}}

In [17]:
with open('../data/mingkh_addresses_parsed.json', 'w') as fh:
    json.dump(mingkh_addresses, fh, ensure_ascii=False)

In [19]:
mos_ru_filtered_df = pd.read_json(DATA_DIR / 'mos_ru_filtered_data.json')

In [35]:
def try_fix_mos_ru_address(address):
    address = address.replace('-й', '')
    for street_type in ('пер.', 'проезд', 'б-р', 'просек', 'проспект', 'переулок', 'улица', 'тупик'):
        if street_type in address:
            address = street_type + ' ' + address.replace(street_type, '')
    return address

In [39]:
mos_ru_filtered_df[mos_ru_filtered_df.SIMPLE_ADDRESS.apply(lambda x: 'Нагатинская набережная, дом 48' in x)]

,global_id,DISTRICT,SIMPLE_ADDRESS,ADM_AREA,geodata_center
22743,646093616,муниципальный округ Нагатинский Затон,"Нагатинская набережная, дом 48/2, строение 3",Южный административный округ,"[37.680073722, 55.687194998]"
62597,647389699,муниципальный округ Нагатинский Затон,"Нагатинская набережная, дом 48/2, строение 2",Южный административный округ,"[37.680601813, 55.687220458]"
98770,648954930,муниципальный округ Нагатинский Затон,"Нагатинская набережная, дом 48/2",Южный административный округ,"[37.679221432, 55.687336217]"


In [60]:
parser('Нагатинская наб., дом 48/2')

[{'settlement': 'нагатинская',
  'street_type': 'наб',
  'house_type': 'дом',
  'house': '48/2',
  'unparsed_parts': []}]

In [55]:
parser('нагатинская наб., 48/2')

[{'settlement': 'нагатинская',
  'street_type': 'наб',
  'house': '48/2',
  'unparsed_parts': []}]

In [68]:
list(map(lambda x: x['parsed_mingkh_address']['street_type'], (filter(lambda x: 'street_type' in x['parsed_mingkh_address'] , mingkh_addresses))))

['улица',
 'улица',
 'улица',
 'проезд',
 'улица',
 'улица',
 'проспект',
 'улица',
 'проезд',
 'улица',
 'б-р',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'шоссе',
 'улица',
 'улица',
 'переулок',
 'улица',
 'улица',
 'улица',
 'проспект',
 'улица',
 'улица',
 'наб',
 'улица',
 'улица',
 'шоссе',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'шоссе',
 'улица',
 'улица',
 'улица',
 'переулок',
 'улица',
 'улица',
 'улица',
 'улица',
 'линия',
 'улица',
 'улица',
 'улица',
 'проспект',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'б-р',
 'улица',
 'переулок',
 'переулок',
 'проспект',
 'проезд',
 'улица',
 'проспект',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'проезд',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'проспект',
 'улица',
 'проспект',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица',
 'проезд',
 'улица',
 'улица',
 'улица',
 'улица',
 'улица'

In [37]:
n_errors = 0
n_fixed_errors = 0
N_TEST_ADDRESSES = mos_ru_filtered_df.shape[0]
for row in tqdm(mos_ru_filtered_df.iloc[:N_TEST_ADDRESSES].itertuples(), total=N_TEST_ADDRESSES):
    address = row.SIMPLE_ADDRESS
    
    parsed_address = parser(address)[0]
    unparsed = parsed_address['unparsed_parts']
    if len(unparsed) > 0:
        n_errors += 1
        fixed_address = try_fix_mos_ru_address(address)
        parsed_address = parser(fixed_address)[0]
        unparsed = parsed_address['unparsed_parts']
        if len(unparsed) == 0:
            n_fixed_errors += 1
        else:
            print(f'{address} | {unparsed}')
            
print(f'\nTotal {n_errors} unparsed (total {float(n_errors) / N_TEST_ADDRESSES * 100:0.2f}%, total {float(n_fixed_errors) / n_errors * 100:0.2f}%)')

  0%|          | 0/126996 [00:00<?, ?it/s]

проспект Вернадского, дом 100, корпус ЗОНА, строение В-1-2-3 | ['строение']


KeyboardInterrupt: 

In [ ]:
n_errors = 0
n_fixed_errors = 0
N_TEST_ADDRESSES = len(addresses)
for address in tqdm(addresses[:N_TEST_ADDRESSES]):
    parsed_address = parser(address)[0]
    unparsed = parsed_address['unparsed_parts']
    if len(unparsed) > 0:
        n_errors += 1
        fixed_address = try_fix_mingkh_address(address)
        parsed_address = parser(fixed_address)[0]
        unparsed = parsed_address['unparsed_parts']
        if len(unparsed) == 0:
            n_fixed_errors += 1
        else:
            print(f'{address} | {unparsed}')
        
print(f'\nTotal {n_errors} unparsed (total {float(n_errors) / N_TEST_ADDRESSES * 100:0.2f}%, total {float(n_fixed_errors) / n_errors * 100:0.2f}%)')